# Segment stitching approach

## Preparation

### Imports and authentication

In [1]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 38.7 MB/s eta 0:00:00


In [2]:
from google.colab import auth
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches
import cv2 as cv
import pandas as pd
import geopandas as gpd
import subprocess
import itertools
from tqdm import tqdm
import os
import numba as nb
import hashlib
import time
import rasterio
from rasterio import features
from shapely.geometry import shape, Polygon
plt.style.use("seaborn-v0_8-white")

from google.colab import drive
drive.mount('/content/gdrive')

auth.authenticate_user()

Mounted at /content/gdrive


### Download and prepare metadata file

In [3]:
roi = gpd.read_file("/content/gdrive/MyDrive/field_delineation/tiles_roi.gpkg")

In [4]:
!gsutil cp gs://india_field_delineation/metadata/india_airbus_metadata_all.csv /content/metadata.csv
metadata = pd.read_csv("/content/metadata.csv")
metadata[["unknown0", "unknown1", "unknown2", "z", "x", "y"]] = metadata["DLTile_key"].str.split(":", expand=True)
metadata[["z", "x", "y"]] = metadata[["z", "x", "y"]].astype(int)

Copying gs://india_field_delineation/metadata/india_airbus_metadata_all.csv...
\ [1 files][ 11.4 MiB/ 11.4 MiB]                                                
Operation completed over 1 objects/11.4 MiB.                                     


In [6]:
metadata[metadata.image_id == 0]

,tile_id,DLTile_key,image_id,airbus_acquired_date,batch,unknown0,unknown1,unknown2,z,x,y
0,90478,3840:128:1.5:46:47:516,0,2020-11-07T03:52:08.377000+00:00,90,3840,128,1.5,46,47,516
2,90479,3840:128:1.5:46:46:516,0,2020-11-07T03:52:08.377000+00:00,90,3840,128,1.5,46,46,516
4,90480,3840:128:1.5:46:45:515,0,2020-11-07T03:52:08.377000+00:00,90,3840,128,1.5,46,45,515
8,90481,3840:128:1.5:46:45:516,0,2020-11-07T03:52:08.377000+00:00,90,3840,128,1.5,46,45,516
12,90482,3840:128:1.5:46:44:515,0,2020-11-07T03:52:08.377000+00:00,90,3840,128,1.5,46,44,515
...,...,...,...,...,...,...,...,...,...,...,...
177115,62031,3840:128:1.5:44:-52:372,0,2020-11-05T04:57:03.741000+00:00,62,3840,128,1.5,44,-52,372
177116,62033,3840:128:1.5:44:-52:373,0,2020-11-05T04:57:03.741000+00:00,62,3840,128,1.5,44,-52,373
177117,62042,3840:128:1.5:44:-54:370,0,2020-11-05T04:57:03.741000+00:00,62,3840,128,1.5,44,-54,370
177119,62048,3840:128:1.5:44:-55:370,0,2020-11-05T04:57:03.741000+00:00,62,3840,128,1.5,44,-55,370


In [ ]:
#meta_df = metadata.loc[(metadata.DLTile_key.isin(roi.key)) & (metadata.image_id==0)]

In [ ]:
meta_df = pd.merge(roi, metadata, how="left", left_on="key", right_on="DLTile_key")
meta_df = meta_df.loc[meta_df.image_id==0]
#meta_df = meta_df.loc[(meta_df.x < -41) & (meta_df.y < 436)]
meta_df

,key,geometry,tile_id,DLTile_key,image_id,airbus_acquired_date,batch,unknown0,unknown1,unknown2,z,x,y
0,3840:128:1.5:44:-44:433,"POLYGON ((78.53404 22.53274, 78.59374 22.53364...",72472,3840:128:1.5:44:-44:433,0,2020-11-05T04:55:10.232000+00:00,72,3840,128,1.5,44,-44,433
2,3840:128:1.5:44:-43:433,"POLYGON ((78.59001 22.53359, 78.64971 22.53447...",72473,3840:128:1.5:44:-43:433,0,2020-11-05T04:55:10.232000+00:00,72,3840,128,1.5,44,-43,433
4,3840:128:1.5:44:-42:433,"POLYGON ((78.64597 22.53442, 78.70567 22.53528...",72474,3840:128:1.5:44:-42:433,0,2020-11-05T04:55:10.232000+00:00,72,3840,128,1.5,44,-42,433
5,3840:128:1.5:44:-41:433,"POLYGON ((78.70194 22.53522, 78.76165 22.53607...",72475,3840:128:1.5:44:-41:433,0,2020-11-05T04:55:10.232000+00:00,72,3840,128,1.5,44,-41,433
6,3840:128:1.5:44:-40:433,"POLYGON ((78.75791 22.53601, 78.81762 22.53684...",72476,3840:128:1.5:44:-40:433,0,2020-11-05T04:55:10.232000+00:00,72,3840,128,1.5,44,-40,433
...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,3840:128:1.5:44:-39:442,"POLYGON ((78.80642 23.00475, 78.86633 23.00557...",72612,3840:128:1.5:44:-39:442,0,2020-11-05T04:55:10.232000+00:00,72,3840,128,1.5,44,-39,442
180,3840:128:1.5:44:-38:442,"POLYGON ((78.86259 23.00552, 78.92250 23.00631...",72613,3840:128:1.5:44:-38:442,0,2020-11-05T04:55:10.232000+00:00,72,3840,128,1.5,44,-38,442
184,3840:128:1.5:44:-37:442,"POLYGON ((78.91876 23.00627, 78.97867 23.00704...",72614,3840:128:1.5:44:-37:442,0,2020-11-05T04:55:10.232000+00:00,72,3840,128,1.5,44,-37,442
188,3840:128:1.5:44:-36:442,"POLYGON ((78.97493 23.00699, 79.03485 23.00775...",72615,3840:128:1.5:44:-36:442,0,2020-10-03T05:00:10.865000+00:00,72,3840,128,1.5,44,-36,442


### Download the images

### Load the images

In [ ]:
watershed_imgs = {}
instance_uncertainty_imgs = {}
semantic_uncertainty_imgs = {}

for i, row in tqdm(meta_df.iterrows()):
    tile_id = row["tile_id"]
    batch = row["batch"]
    img_path = "/content/gdrive/MyDrive/field_delineation/imgs/watershed_tileable/{:02d}_{:05d}.npy".format(batch, tile_id)
    if os.path.exists(img_path):
        watershed_imgs["{:02d}_{:05d}".format(batch, tile_id)] = np.load(img_path)
    img_path = "/content/gdrive/MyDrive/field_delineation/imgs/instance_uncertainty/{:02d}_{:05d}.png".format(batch, tile_id)
    if os.path.exists(img_path):
        instance_uncertainty_imgs["{:02d}_{:05d}".format(batch, tile_id)] = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
    img_path = "/content/gdrive/MyDrive/field_delineation/imgs/semantic_uncertainty/{:02d}_{:05d}.png".format(batch, tile_id)
    if os.path.exists(img_path):
        semantic_uncertainty_imgs["{:02d}_{:05d}".format(batch, tile_id)] = cv.imread(img_path, cv.IMREAD_GRAYSCALE)

34it [01:49,  4.38s/it]

In [ ]:
#meta_df_old = metadata.loc[(metadata.image_id == 0) & (metadata.batch == 0) & (metadata.z==42) & (metadata.x >= 50) & (metadata.x < 54) & (metadata.y >= 408) & (metadata.y < 412)]

In [ ]:
def draw_segmentation(img, mapping="random", ax=None):
    if mapping is None:
        draw_img = img
        field_id_mapping = None
    else:
        if mapping == "random":
            field_idxs = np.unique(img)
            np.random.shuffle(field_idxs)
            field_id_mapping = {field_id: i for i,
                            field_id in enumerate(field_idxs)}
        else:
            field_id_mapping = mapping
        draw_img = np.vectorize(field_id_mapping.get)(img)
        draw_img[draw_img == None] = 0
        draw_img = draw_img.astype(np.int64)
        draw_img = np.ma.masked_where(img<2, draw_img)

    cmap = plt.get_cmap('tab20', np.max(draw_img))
    if ax is None:
        fig, ax = plt.subplots()
    ax.imshow(draw_img, cmap="tab20", interpolation="none")
    return field_id_mapping

In [ ]:
"""
fig, axs = plt.subplots(4, 4, figsize=(15, 15))
for i, (key, img) in tqdm(enumerate(watershed_imgs.items())):
    batch, tile_id = key.split("_")
    tile_id = int(tile_id)
    row = meta_df_old.loc[meta_df_old["tile_id"] == tile_id].iloc[0]
    c0 = int(row["z"])
    c1 = int(row["x"])
    c2 = int(row["y"])

    x = c1-50
    y = 411-c2

    if x not in range(4) or y not in range(4):
        continue
    ax = axs[y][x]

    coords_str = "{}: [{}, {}, {}]".format(key, c0, c1, c2)
    draw_segmentation(img, ax=ax)
    #ax.imshow(img)
    ax.set_title(coords_str)
    ax.axis("off")
"""
pass

In [ ]:
watershed_imgs.keys()

dict_keys(['72_72472', '72_72473', '72_72474', '72_72487', '72_72488', '72_72489', '72_72502', '72_72503', '72_72504'])

In [ ]:
MIN_SEGMENT_SIZE = 222
MAX_ID = 2**16-1

def filter_and_reindex(img):
    img = img.copy()
    ids, counts = np.unique(img, return_counts=True)
    img[np.isin(img, ids[counts < MIN_SEGMENT_SIZE])] = 0

    ids = np.unique(img)
    return _filter_and_reindex(img, ids)

def _filter_and_reindex(img, ids):
    # = np.zeros(img.shape[0]*img.shape[1], dtype=np.uint16)
    res_img = np.ravel(img)
    #res_img = flatten_img.copy()
    j = 0
    free_ids = np.arange(MAX_ID)
    free_ids = free_ids[~np.isin(free_ids, ids)]
    too_high_ids = ids[ids > MAX_ID]
    print("\nReindexing {} segments".format(len(too_high_ids)))
    for id in too_high_ids:
        res_img[res_img == id] = free_ids[j]
        j += 1
        if j >= len(free_ids):
            raise("not enough free ids in int16")
    return res_img.reshape(img.shape)

In [ ]:
#transform = rasterio.transform.from_origin(0, 0, 1, 1)  # Example raster transform

# Create a generator of polygons from the raster segments
def raster_to_polygons(raster):
    for geom, value in features.shapes(raster):
        yield shape(geom), value

def raster_to_gdf(raster):
    raster = filter_and_reindex(raster)
    # Convert raster segments to vector polygons
    polygons = []
    for polygon, value in raster_to_polygons(raster.astype(np.uint16)):
        if value != 0:  # Skip polygons where value is 0
            polygons.append({'geometry': polygon, 'properties': {'value': int(value)}})

    # Convert polygons to GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(polygons)
    return gdf

In [ ]:
#def map_square_to_rotated_rectangle(rectangle_coords, polygons):
    #square_coords = [(4096, 0), (4096, 4096), (0, 4096), (0, 0)]

def map_points(geoseries, transformed_corners):
    original_corners = [(0, 4096), (4096, 4096), (4096, 0), (0, 0)]

    # Calculate transformation matrices
    A = np.column_stack((transformed_corners[:, ::-1], np.ones(len(transformed_corners))))
    B = np.column_stack((original_corners, np.ones(len(original_corners))))
    T = np.linalg.lstsq(B, A, rcond=None)[0]

    # Apply transformation to each polygon in the GeoSeries
    transformed_geometries = []
    for polygon in geoseries.geometry:
        transformed_exterior = []
        for point in polygon.exterior.coords:
            extended_point = [point[0], point[1], 1]  # Extend point to include homogeneous coordinate
            transformed_point = np.dot(extended_point, T)
            transformed_exterior.append((transformed_point[1], transformed_point[0]))
        transformed_polygon = Polygon(transformed_exterior)
        transformed_geometries.append(transformed_polygon)

    return gpd.GeoSeries(transformed_geometries)

In [ ]:
def sort_quad(polygon):
    # Create a Shapely polygon from the coordinates
    polygon_coords = np.array(polygon.exterior.coords)[:-1, :]

    # Calculate the centroid of the polygon
    centroid = polygon.centroid

    # Get the centroid coordinates
    centroid_x, centroid_y = centroid.x, centroid.y

    # Create a list to store the vertices categorized by their positions
    sorted_vertices = [] #{'lower_left': [], 'lower_right': [], 'upper_left': [], 'upper_right': []}

    # Iterate through each vertex and categorize them based on their positions relative to the centroid
    for vertex in polygon_coords:
        x, y = vertex
        if x < centroid_x and y < centroid_y:
            sorted_vertices.append(vertex)
        elif x > centroid_x and y < centroid_y:
            sorted_vertices.append(vertex)
        elif x > centroid_x and y > centroid_y:
            sorted_vertices.append(vertex)
        elif x < centroid_x and y > centroid_y:
            sorted_vertices.append(vertex)


    return np.array(sorted_vertices)

In [ ]:
meta_df_subsample = meta_df#meta_df.loc[(meta_df.x < -42) & (meta_df.y < 435)]

In [ ]:
gdfs_transformed = []

for j, (i, row) in tqdm(enumerate(meta_df_subsample.iterrows())):
    tile_id = row["tile_id"]
    batch = row["batch"]
    file_id = "{:02d}_{:05d}".format(batch, tile_id)
    tile_geometry = row.geometry
    #draw_segmentation(watershed_imgs[file_id])
    gdf = raster_to_gdf(watershed_imgs[file_id])
    centroid_coords = np.array([(point.x, point.y) for point in gdf.geometry.centroid])
    centroid_coords = centroid_coords.astype(int)
    gdf["instance_uncertainty"] = instance_uncertainty_imgs[file_id][centroid_coords[:, 0], centroid_coords[:, 1]] / 255.0
    gdf["semantic_uncertainty"] = semantic_uncertainty_imgs[file_id][centroid_coords[:, 0], centroid_coords[:, 1]] / 255.0

    gdf_transformed = gdf.copy()
    gdf_transformed.geometry = map_points(gdf.geometry, sort_quad(tile_geometry))
    gdfs_transformed.append(gdf_transformed)
    #gdf_transformed.geometry.plot(ax=ax)
    #ax.plot(np.array(tile_geometry.exterior.xy[0]), np.array(tile_geometry.exterior.xy[1]), color="red")

0it [00:00, ?it/s]


Reindexing 5948 segments


1it [01:24, 84.95s/it]


Reindexing 5177 segments


2it [02:39, 78.69s/it]

...

98it [1:20:33, 19.77s/it]


Reindexing 2134 segments


99it [1:21:18, 27.50s/it]


Reindexing 5344 segments


100it [1:22:42, 49.62s/it]


In [ ]:
"""
fig, ax = plt.subplots(1, figsize=(15, 12))

for j, (i, row) in tqdm(enumerate(meta_df_subsample.iterrows())):
    tile_id = row["tile_id"]
    batch = row["batch"]
    tile_geometry = row.geometry
    gdfs_transformed[j].plot(ax=ax, column="instance_uncertainty", cmap="viridis", vmin=0, vmax=1, legend=j==0)
    ax.plot(np.array(tile_geometry.exterior.xy[0]), np.array(tile_geometry.exterior.xy[1]), color="red", lw="2")
    ax.grid(True)
    if j>2:
        break

ax.set_title("Vectorized segments > 500m2, colored by instance uncertainty")
fig.tight_layout()
"""
pass

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#fig.savefig("/content/gdrive/MyDrive/segments_colored4x4.png", dpi=300)

In [ ]:
gdf_total = pd.concat(gdfs_transformed)

In [ ]:
gdf_total.drop(columns="value", inplace=True)

In [ ]:
#gdf_total.to_file("/content/gdrive/MyDrive/field_delineation/fields_mp_10x10.gpkg", layer='fields', driver="GPKG")
gdf_total.to_file("/content/gdrive/MyDrive/field_delineation/fields_mp_10x10.geojson", driver="GeoJSON")

In [ ]:
gdf_total.to_file("/content/gdrive/MyDrive/field_delineation/fields_mp_10x10.shp")

<ipython-input-176-cd61f6f77167>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_total.to_file("/content/gdrive/MyDrive/field_delineation/fields_mp_10x10.shp")


In [ ]:
gdf_total.to_feather("/content/gdrive/MyDrive/field_delineation/fields_mp_10x10.feather")

In [ ]:
gdf_total.head(1000).to_file("/content/gdrive/MyDrive/field_delineation/fields_mp_10x10_sample1000.geojson", driver="GeoJSON")

In [ ]:
gdf_total

,geometry,instance_uncertainty,semantic_uncertainty
0,"POLYGON ((78.56274 22.58696, 78.56274 22.58695...",0.690196,0.505882
1,"POLYGON ((78.56140 22.58709, 78.56140 22.58693...",0.654902,0.427451
2,"POLYGON ((78.53992 22.58700, 78.53994 22.58700...",0.376471,0.654902
3,"POLYGON ((78.58525 22.58751, 78.58526 22.58751...",0.666667,0.478431
4,"POLYGON ((78.55997 22.58718, 78.56000 22.58718...",0.388235,0.529412
...,...,...,...
13916,"POLYGON ((79.03385 23.00948, 79.03391 23.00948...",0.031373,0.894118
13917,"POLYGON ((79.03916 23.00978, 79.03917 23.00978...",0.000000,0.807843
13918,"POLYGON ((79.04178 23.00976, 79.04181 23.00976...",0.000000,0.929412
13919,"POLYGON ((79.05312 23.00973, 79.05318 23.00973...",0.266667,0.466667
